In [10]:
## IMPORT MODULES
import pandas as pd
import numpy as np
import os,time,fnmatch

In [11]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
%matplotlib widget


In [12]:
## CREATE LINK TO DATAPATH MODULE
import sys
sys.path.insert(0,'/home/bhabas/catkin_ws/src/crazyflie_simulation/src/crazyflie_gazebo_sim/src/utility')
from data_analysis import DataFile
dataPath = '/home/bhabas/catkin_ws/src/crazyflie_simulation/local_files/data/'

In [13]:
## CREATE AN ARRAY FOR ALL TEST COMBINATIONS
temp_list = []
for vz_d in np.arange(1.5,4.0,0.5): # [1.5,3.5]
    for vx_d in np.arange(0,3.5,0.5):   # [0,3.0]
        # print(f"{vz_d}|{vx_d}")
        temp_list.append([vz_d,vx_d])
test_arr = np.asarray(temp_list)
test_arr

array([[1.5, 0. ],
       [1.5, 0.5],
       [1.5, 1. ],
       [1.5, 1.5],
       [1.5, 2. ],
       [1.5, 2.5],
       [1.5, 3. ],
       [2. , 0. ],
       [2. , 0.5],
       [2. , 1. ],
       [2. , 1.5],
       [2. , 2. ],
       [2. , 2.5],
       [2. , 3. ],
       [2.5, 0. ],
       [2.5, 0.5],
       [2.5, 1. ],
       [2.5, 1.5],
       [2.5, 2. ],
       [2.5, 2.5],
       [2.5, 3. ],
       [3. , 0. ],
       [3. , 0.5],
       [3. , 1. ],
       [3. , 1.5],
       [3. , 2. ],
       [3. , 2.5],
       [3. , 3. ],
       [3.5, 0. ],
       [3.5, 0.5],
       [3.5, 1. ],
       [3.5, 1.5],
       [3.5, 2. ],
       [3.5, 2.5],
       [3.5, 3. ]])

In [14]:
## CREATE DATAFRAME FROM ALL TESTS

d = []                                                  # Create empty list
for vz_d,vx_d in test_arr:                              # Iter over all combinations in test_arr
    for file in os.listdir(dataPath):                   # Iter over all files in dir 
        csv = os.path.join(dataPath,file)               #   and join name w/ base path

        if fnmatch.fnmatch(file,f"Vz_{vz_d}--Vx_{vx_d}--trial_*.csv"): # If filename matched selected IC 

            trial = DataFile(csv)                       # Create object called trial

            trial_num = file[-5]                        # Retrieve trial num
            landing_rate = trial.landing_rate()         # Retrieve landing success rate
            policy,sigma = trial.grab_finalPolicy()     # Retrieve final policy and sigma
            omega_d = trial.grab_omega_d_trial()
            omega_flip = trial.grab_omega_flip_trial()
            vel_flip = trial.grab_vel_flip_trial()

            RREV_trigger,G1,G2 = policy                 # Break up policy
            RREV_sig,G1_sig,G2_sig = sigma              # Break up sigma
            wx_d,wy_d,wz_d = omega_d
            wx_f,wy_f,wz_f = omega_flip
            vx_f,vy_f,vz_f = vel_flip


            d.append((vz_d,vx_d,trial_num,landing_rate, # Append values to list
                RREV_trigger,G1,G2,
                RREV_sig,G1_sig,G2_sig,
                wx_d,wy_d,wz_d,
                wx_f,wy_f,wz_f,
                vx_f,vy_f,vz_f))

policy_df = pd.DataFrame(d, columns=( # Convert list to pd.dataframe
    'vz_d','vx_d','trial_num','landing_rate',
    'RREV_trigger','G1','G2',
    'RREV_sig','G1_sig','G2_sig',
    'wx_d','wy_d','wz_d',
    'wx_f','wy_f','wz_f',
    'vx_f','vy_f','vz_f'))
policy_df

,vz_d,vx_d,trial_num,landing_rate,RREV_trigger,G1,G2,RREV_sig,G1_sig,G2_sig,wx_d,wy_d,wz_d,wx_f,wy_f,wz_f,vx_f,vy_f,vz_f
0,1.5,0.0,1,0.000000,4.11,6.18,3.78,0.07,0.38,0.28,0.0,1.594084,0.0,-0.051048,1.261809,-2.016673,0.023086,-0.008365,1.083670
1,1.5,0.0,0,0.000000,5.00,7.59,3.88,0.83,0.27,0.11,0.0,5.917062,0.0,-0.567061,-2.006484,-1.838774,-0.072994,0.098063,0.824812
2,1.5,0.0,2,0.000000,4.82,5.82,3.44,0.08,0.15,0.28,0.0,6.118875,0.0,-0.573339,2.427546,-2.339939,0.045777,0.013838,0.857656
3,1.5,0.5,2,0.000000,4.33,7.39,3.25,0.10,0.19,0.07,0.0,-26.630638,0.0,0.108152,-16.009908,-3.229212,0.506836,-0.005285,1.166715
4,1.5,0.5,1,0.000000,4.55,6.98,3.86,0.13,0.10,0.14,0.0,-24.938559,0.0,-0.118284,-15.512765,-2.237243,0.480374,0.005073,1.101052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,3.5,2.5,1,0.750000,5.16,7.81,1.81,0.16,0.05,0.52,0.0,-33.333030,0.0,-0.528365,-23.278037,-2.163531,2.266449,-0.016500,2.702471
101,3.5,2.5,2,0.350000,4.82,8.09,1.71,0.08,0.23,0.17,0.0,-33.203274,0.0,-0.072532,-20.304746,-1.529446,2.178745,-0.015988,2.567516
102,3.5,3.0,2,0.631579,5.16,8.27,4.02,0.13,0.07,0.01,0.0,-22.577884,0.0,0.271152,-24.562926,-1.931681,2.575116,-0.059825,2.453206
103,3.5,3.0,1,0.947368,5.12,6.52,0.61,0.03,0.20,0.29,0.0,-31.344009,0.0,-2.375304,-23.671941,-0.412886,2.531474,-0.078620,2.425725


In [15]:
policy_df

,vz_d,vx_d,trial_num,landing_rate,RREV_trigger,G1,G2,RREV_sig,G1_sig,G2_sig,wx_d,wy_d,wz_d,wx_f,wy_f,wz_f,vx_f,vy_f,vz_f
0,1.5,0.0,1,0.000000,4.11,6.18,3.78,0.07,0.38,0.28,0.0,1.594084,0.0,-0.051048,1.261809,-2.016673,0.023086,-0.008365,1.083670
1,1.5,0.0,0,0.000000,5.00,7.59,3.88,0.83,0.27,0.11,0.0,5.917062,0.0,-0.567061,-2.006484,-1.838774,-0.072994,0.098063,0.824812
2,1.5,0.0,2,0.000000,4.82,5.82,3.44,0.08,0.15,0.28,0.0,6.118875,0.0,-0.573339,2.427546,-2.339939,0.045777,0.013838,0.857656
3,1.5,0.5,2,0.000000,4.33,7.39,3.25,0.10,0.19,0.07,0.0,-26.630638,0.0,0.108152,-16.009908,-3.229212,0.506836,-0.005285,1.166715
4,1.5,0.5,1,0.000000,4.55,6.98,3.86,0.13,0.10,0.14,0.0,-24.938559,0.0,-0.118284,-15.512765,-2.237243,0.480374,0.005073,1.101052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,3.5,2.5,1,0.750000,5.16,7.81,1.81,0.16,0.05,0.52,0.0,-33.333030,0.0,-0.528365,-23.278037,-2.163531,2.266449,-0.016500,2.702471
101,3.5,2.5,2,0.350000,4.82,8.09,1.71,0.08,0.23,0.17,0.0,-33.203274,0.0,-0.072532,-20.304746,-1.529446,2.178745,-0.015988,2.567516
102,3.5,3.0,2,0.631579,5.16,8.27,4.02,0.13,0.07,0.01,0.0,-22.577884,0.0,0.271152,-24.562926,-1.931681,2.575116,-0.059825,2.453206
103,3.5,3.0,1,0.947368,5.12,6.52,0.61,0.03,0.20,0.29,0.0,-31.344009,0.0,-2.375304,-23.671941,-0.412886,2.531474,-0.078620,2.425725


In [16]:
## POLICY AVERAGED DATAFRAME

# Group dataframe values by IC and find avg/std of their values 
policy_avg_df = policy_df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()
policy_avg_df


vz_d vx_d landing_rate           RREV_trigger                   G1  \
                     mean       std         mean       std       mean   
0   1.5  0.0     0.000000  0.000000     4.643333  0.470567   6.530000   
1   1.5  0.5     0.000000  0.000000     4.376667  0.155349   7.506667   
2   1.5  1.0     0.022222  0.038490     4.726667  0.513939   6.460000   
3   1.5  1.5     0.000000  0.000000     4.460000  0.200000  10.226667   
4   1.5  2.0     0.000000  0.000000     3.220000  0.644903   3.470000   
5   1.5  2.5     0.000000  0.000000     2.333333  0.265016   5.586667   
6   1.5  3.0          NaN       NaN     2.846667  1.591048   5.870000   
7   2.0  0.0     0.027778  0.048113     5.120000  0.756637   6.483333   
8   2.0  0.5     0.000000  0.000000     4.436667  0.536315   6.936667   
9   2.0  1.0     0.215278  0.138715     4.966667  0.209841   8.013333   
10  2.0  1.5     0.336601  0.161789     4.850000  0.230000   7.793333   
11  2.0  2.0     0.425000  0.066144     5.036667  0.161658  10.813333   
12  2.0  2.5     0.385281  0.304665     4.843333  0.232451   5.470000   
13  2.0  3.0     0.178205  0.178951     5.023333  0.335012   3.800000   
14  2.5  0.0     0.083333  0.028868     5.250000  0.055678   7.016667   
15  2.5  0.5     0.203509  0.006077     5.100000  0.176918   6.890000   
16  2.5  1.0     0.728947  0.125187     5.316667  0.095044   7.316667   
17  2.5  1.5     0.833333  0.104083     5.400000  0.017321   7.690000   
18  2.5  2.0     0.833333  0.152753     5.620000  0.175784   8.670000   
19  2.5  2.5     0.695614  0.326955     5.043333  0.372335   8.616667   
20  2.5  3.0     0.744444  0.168600     5.193333  0.030551   6.403333   
21  3.0  0.0     0.051754  0.052654     5.173333  0.056862   6.240000   
22  3.0  0.5     0.500000  0.100000     4.893333  0.176163   7.400000   
23  3.0  1.0     0.816667  0.144338     4.903333  0.085049   7.743333   
24  3.0  1.5     0.880702  0.060144     5.113333  0.200333   6.626667   
25  3.0  2.0     0.783333  0.104083     5.230000  0.174356   8.813333   
26  3.0  2.5     0.770955  0.258641     5.266667  0.200083   8.463333   
27  3.0  3.0     0.647563  0.046550     5.370000  0.180278   7.316667   
28  3.5  0.0     0.483333  0.104083     4.673333  0.153080   7.763333   
29  3.5  0.5     0.600000  0.086603     4.806667  0.081445   8.270000   
30  3.5  1.0     0.712281  0.054336     4.923333  0.151767   6.903333   
31  3.5  1.5     0.900000  0.050000     5.230000  0.121655  11.450000   
32  3.5  2.0     0.883333  0.160728     5.133333  0.277549   9.190000   
33  3.5  2.5     0.664912  0.282160     4.923333  0.205508   7.490000   
34  3.5  3.0     0.859649  0.199260     5.093333  0.083267   8.306667   

                    G2            ...       wy_f                 wz_f  \
         std      mean       std  ...       mean        std      mean   
0   0.935468  3.700000  0.230651  ...   0.560957   2.298598 -2.065129   
1   0.593661  2.800000  1.342796  ... -16.084634   0.612660 -2.799901   
2   5.671517  3.246667  0.875233  ...  -5.491657  16.028165 -1.894050   
3   8.137840  1.370000  1.988492  ... -15.575749   0.994363 -2.003301   
4   2.911563  7.646667  3.037636  ...  15.785670   3.233447 -1.734951   
5   2.476900  6.976667  3.007396  ...  13.027271   3.800369 -1.140795   
6   0.153948  3.520000  0.854927  ...   5.600963   0.944873  0.094794   
7   1.401761  6.633333  4.784740  ...   1.773749   2.951251 -1.626546   
8   1.269501  1.940000  2.014274  ... -13.292631   1.005389 -2.488981   
9   1.054625  2.950000  0.797308  ... -11.292908   1.619217 -0.993884   
10  1.029093  0.826667  0.265769  ... -13.264592   2.349401 -1.175714   
11  4.143445  6.813333  5.565270  ...  -4.790506  15.978690 -0.883217   
12  4.060702  3.916667  4.900228  ...  -5.608518  14.862671 -2.106162   
13  2.131267  5.763333  0.884609  ...   8.228886   5.297130 -3.690364   
14  1.159411  3.443333  1.079367  ... -18.938304   1.099854 -2.377214   
15  1.049714  2.050000  1.299654  ... -17.646727   0.578895 

# Landing Rate Data

In [17]:
## LANDING RATE DATA
LR_df = policy_avg_df[['vz_d','vx_d','landing_rate']]
# LR_df

In [18]:
# LR_df['landing_rate']
LR_df['landing_rate','mean']


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.plot_trisurf(LR_df['vx_d'],LR_df['vz_d'],LR_df['landing_rate','mean'],cmap=cm.jet)

ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('Avg LR [%]')
ax.set_title('Average Landing Rate for Final Two Episodes')



ax.yaxis.set_ticks(np.arange(1.5,4.0,0.5))
fig.tight_layout()
plt.show()

## Disregard any data Vz_d < 2.5

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
## LANDING RATE PER TRIAL

fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['landing_rate'])

ax.set_title('Landing Rate per Trial (Raw)')
ax.set_xlabel('$V_{zd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('LR %')

ax.yaxis.set_ticks(np.arange(1.5,4.0,0.5))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
## DROP VZ_D < 2.5 M/S
policy_df = policy_df[ policy_df['vz_d'] >= 2.5]                # Only keep rows w/ vz_d >= 2.5 m/s
policy_avg_df = policy_avg_df[ policy_avg_df['vz_d'] >= 2.5]    # Only keep rows w/ vz_d >= 2.5 m/s

# policy_avg_df

## RREV_trigger Avg(3D)

In [21]:
# RREV Data

In [22]:
## RREV_trigger Average (3D)

In [23]:
## RREV_TRIGGER DATA
RREV_df = policy_avg_df[['vz_d','vx_d','RREV_trigger']]
## RREV_TRIGGER PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(RREV_df['vx_d'],RREV_df['vz_d'],RREV_df['RREV_trigger','mean'],color='red')


ax.set_title('Avg RREV_trigger per Trial')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('RREV')

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
ax.set_zlim(0)
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## RREV_trigger Raw (3D)

In [24]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['RREV_trigger'])

ax.set_title('RREV_trigger per Trial (Raw)')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('RREV')
ax.set_zlim(0)
ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## RREV_trigger Std (2D)

In [25]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['RREV_trigger','std']
    ax.errorbar(df['vx_d'],df['RREV_trigger','mean'],yerr=2*std,label='RREV_trigger (2$\sigma$)',linestyle='None',marker='o', capsize=5, capthick=1, ecolor='black')

    ax.set_ylim(0,10)
    ax.set_xlabel('$V_{xd} \ [m/s]$')
    ax.set_ylabel('RREV [rad/s]')
    ax.set_title(f'RREV vs Vx_d | Vz_d = {vz} [m/s]')
    ax.grid()
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# G1 Data

In [26]:
## G1 AVG DATA
G1_df = policy_avg_df[['vz_d','vx_d','G1']]
G1_df


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(G1_df['vx_d'],G1_df['vz_d'],G1_df['G1','mean'],color='red')
ax.set_title('G1 per Trial (Avg)')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('G1')
ax.set_zlim(0)

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['G1'])

ax.set_title('G1 per Trial (Raw)')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('G1')
ax.set_zlim(0)

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['G1','std']
    ax.errorbar(df['vx_d'],df['G1','mean'],yerr=2*std,label='G1 - mean (2$\sigma$)',linestyle='None',marker='o', capsize=5, capthick=1, ecolor='black')


    ax.set_xlabel('$V_{xd} \ [m/s]$')
    ax.set_ylabel('G1')
    ax.set_title(f'G1 vs Vx_d | Vz_d = {vz} [m/s]')

    ax.set_ylim(0,20)
    ax.grid()
    ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# G2 Data

In [29]:
## G2 AVG DATA
G2_df = policy_avg_df[['vz_d','vx_d','G2']]
G2_df


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(G2_df['vx_d'],G2_df['vz_d'],G2_df['G2','mean'],color='red')
ax.set_title('G2 per Trial (Avg)')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('G2')

ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['G2'])

ax.set_title('G2 per Trial (Raw)')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('G2')

ax.set_zlim(0,15)
ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['G2','std']
    ax.errorbar(df['vx_d'],df['G2','mean'],yerr=2*std,label='G2 (2$\sigma$)',linestyle='None',marker='o', capsize=5, capthick=1, ecolor='black')


    ax.set_xlabel('$V_{xd} \ [m/s]$')
    ax.set_ylabel('G2')
    ax.set_title(f'G2 vs Vx_d | Vz_d = {vz} [m/s]')

    ax.set_ylim(-5,15)
    ax.legend()
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
policy_avg_df.describe()

vz_d       vx_d landing_rate            RREV_trigger             \
                                   mean        std         mean        std   
count  21.00000  21.000000    21.000000  21.000000    21.000000  21.000000   
mean    3.00000   1.500000     0.651286   0.124576     5.126508   0.146239   
std     0.41833   1.024695     0.254360   0.085495     0.221371   0.084783   
min     2.50000   0.000000     0.051754   0.006077     4.673333   0.017321   
25%     2.50000   0.500000     0.600000   0.054336     4.923333   0.083267   
50%     3.00000   1.500000     0.728947   0.104083     5.133333   0.153080   
75%     3.50000   2.500000     0.833333   0.160728     5.250000   0.180278   
max     3.50000   3.000000     0.900000   0.326955     5.620000   0.372335   

              G1                    G2             ...       wy_f             \
            mean        std       mean        std  ...       mean        std   
count  21.000000  21.000000  21.000000  21.000000  ...  21.000000  21.000000   
mean    7.837143   1.417267   2.934286   1.101072  ... -20.477555   1.277525   
std     1.169201   0.804928   1.565435   0.813960  ...   2.736986   0.603667   
min     6.240000   0.526909   0.446667   0.230072  ... -26.088459   0.442791   
25%     7.016667   0.808950   1.980000   0.470850  ... -21.977259   0.767496   
50%     7.690000   1.280989   2.630000   1.079367  ... -19.715646   1.397983   
75%     8.463333   1.805279   4.183333   1.299654  ... -17.993199   1.621288   
max    11.450000   3.899013   6.433333   3.563654  ... -17.180932   2.432981   

            wz_f                  vx_f                  vy_f             \
            mean        std       mean        std       mean        std   
count  21.000000  21.000000  21.000000  21.000000  21.000000  21.000000   
mean   -1.526457   0.625431   1.343128   0.038507  -0.058062   0.039735   
std     0.967091   0.348316   1.003344   0.042288   0.026855   0.025843   
min    -3.106816   0.124081  -0.376333   0.001045  -0.121877   0.003578   
25%    -2.119937   0.360071   0.484400   0.006623  -0.072803   0.019866   
50%    -1.485996   0.632771   1.417367   0.023402  -0.059672   0.036130   
75%    -0.967476   0.828168   2.179215   0.063753  -0.047258   0.054912   
max     0.871668   1.403615   2.850205   0.170181   0.000758   0.106657   

            vz_f             
            mean        std  
count  21.000000  21.000000  
mean    2.444379   0.052182  
std     0.314837   0.056727  
min     2.006293   0.003116  
25%     2.166558   0.012233  
50%     2.434033   0.035525  
75%     2.624011   0.070107  
max     2.927359   0.248555  

[8 rows x 34 columns]

# Omega_d per Trial

In [33]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(policy_df['vx_d'],policy_df['vz_d'],policy_df['wy_d'])

ax.set_title('$\omega_{yd} \ per \ Trial \ (Raw)$')
ax.set_xlabel('$V_{xd} \ [m/s]$')
ax.set_ylabel('$V_{zd} \ [m/s]$')
ax.set_zlabel('$\omega_{yd} \ [rad/s]$')
ax.set_zlim(-60,0)
ax.yaxis.set_ticks(np.arange(2.5,4.0,0.5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    # ax.scatter(df['vx_d'],df['wy_d','mean'],label='$\omega_{yd}$ - mean',marker='o',color='black')
    # ax.scatter(df['vx_d'],df['wy_f','mean'],label='$\omega_{yflip}$ - mean',marker='o',color='red')

    std = df['wy_d','std']
    ax.errorbar(df['vx_d'],df['wy_d','mean'],yerr=2*std,label='$\omega_{yd}$ - mean (2$\sigma$)',
        linestyle='None',
        marker='o', color='black', 
        capsize=5, capthick=1, ecolor='black')

    std = df['wy_f','std']
    ax.errorbar(df['vx_d'],df['wy_f','mean'],yerr=2*std,label='$\omega_{yflip}$ - mean (2$\sigma$)',
        linestyle='None',
        marker='o', color='red', 
        capsize=5, capthick=1, ecolor='red')

    ax.set_xlabel('$V_{xd} \ [m/s]$')
    ax.set_ylabel('$\omega_{yd} \ [rad/s]$')
    ax.set_title(f'Wy_d vs Vx_d | Vz_d = {vz} [m/s]')
    ax.set_ylim(-60,20)
    ax.legend()    
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Vz_actual per Trial

In [35]:
    df = policy_avg_df[policy_avg_df.vz_d==2.5]
    df

vz_d vx_d landing_rate           RREV_trigger                  G1  \
                     mean       std         mean       std      mean   
14  2.5  0.0     0.083333  0.028868     5.250000  0.055678  7.016667   
15  2.5  0.5     0.203509  0.006077     5.100000  0.176918  6.890000   
16  2.5  1.0     0.728947  0.125187     5.316667  0.095044  7.316667   
17  2.5  1.5     0.833333  0.104083     5.400000  0.017321  7.690000   
18  2.5  2.0     0.833333  0.152753     5.620000  0.175784  8.670000   
19  2.5  2.5     0.695614  0.326955     5.043333  0.372335  8.616667   
20  2.5  3.0     0.744444  0.168600     5.193333  0.030551  6.403333   

                    G2            ...       wy_f                wz_f  \
         std      mean       std  ...       mean       std      mean   
14  1.159411  3.443333  1.079367  ... -18.938304  1.099854 -2.377214   
15  1.049714  2.050000  1.299654  ... -17.646727  0.578895 -2.402740   
16  1.280989  2.483333  1.570106  ... -17.675934  2.053385 -1.456663   
17  1.569044  0.446667  0.230072  ... -20.304951  0.560634 -1.942062   
18  2.316225  2.396667  0.757386  ... -23.539372  1.503816 -0.967476   
19  1.288772  0.803333  0.736365  ... -23.046782  1.503681  0.215586   
20  0.526909  0.606667  0.363501  ... -25.021393  2.266199  0.871668   

                  vx_f                vy_f                vz_f            
         std      mean       std      mean       std      mean       std  
14  0.795961 -0.326392  0.037149 -0.047258  0.029784  2.166558  0.006302  
15  1.137589  0.484400  0.014753  0.000758  0.085060  2.136994  0.061591  
16  0.285988  0.984907  0.001045 -0.054742  0.050233  2.174499  0.006705  
17  0.367296  1.474347  0.003414 -0.060576  0.036130  2.157208  0.006023  
18  1.066533  1.970286  0.018388 -0.073406  0.061700  2.157218  0.044044  
19  0.491433  2.355082  0.067542 -0.033078  0.106657  2.006293  0.055829  
20  0.632771  2.850205  0.170181 -0.032487  0.019866  2.017909  0.123992  

[7 rows x 34 columns]

In [36]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    std = df['vz_f','std']

    ax.scatter(df['vx_d'],df['vz_d'],label='$V_{zd}$ - mean',marker='o',color='black')

    # ax.scatter(df['vx_d'],df['vz_f','mean'],label='$V_{zflip}$ mean',marker='o',color='red')
    ax.errorbar(df['vx_d'],df['vz_f','mean'],yerr=2*std,label='$V_{zflip}$ - mean (2$\sigma$)',
        linestyle='None',
        marker='o', color='red', 
        capsize=5, capthick=1, ecolor='black')


    ax.set_xlabel('$V_{xd} \ [m/s]$')
    ax.set_ylabel('$V_{z} \ [m/s]$')
    ax.set_title(f'Vz vs Vx_d | Vz_d = {vz} [m/s]')
    ax.set_ylim(0,5.0)


    ax.legend(loc='upper right')

    
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vx Actual Per Trial

In [37]:
for vz in [2.5,3.0,3.5]:
    df = policy_avg_df[policy_avg_df.vz_d==vz]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    

    ax.scatter(df['vx_d'],df['vx_d'],label='$V_{xd}$ - mean',marker='o',color='black')

    # ax.scatter(df['vx_d'],df['vz_f','mean'],label='$V_{zflip}$ mean',marker='o',color='red')
    std = df['vx_f','std']
    ax.errorbar(df['vx_d'],df['vx_f','mean'],yerr=2*std,label='$V_{xflip}$ - mean (2$\sigma$)',
        linestyle='None',
        marker='o', color='red', 
        capsize=5, capthick=1, ecolor='black')


    ax.set_xlabel('$V_{xd} \ [m/s]$')
    ax.set_ylabel('$V_{x} \ [m/s]$')
    ax.set_title(f'Vx vs Vx_d | Vz_d = {vz} [m/s]')
    ax.set_ylim(0,5.0)


    ax.legend(loc='upper right')

    
    ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RREV vs W_f